<a href="https://colab.research.google.com/github/thursdaydreaming/whitescan/blob/main/FASTTEXT%EB%A1%9C_%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy
!pip install sklearn

In [ ]:
%matplotlib inline
import nltk
nltk.download('punkt')
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt
import csv
import re
import warnings
warnings.filterwarnings("ignore")
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
import pickle
import re
import konlpy
import nltk
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#데이터 불러오기

one = pd.read_excel("drive/MyDrive/large.xlsx", sheet_name='요식유흥')
two = pd.read_excel("drive/MyDrive/large.xlsx", sheet_name='유통')
thr = pd.read_excel("drive/MyDrive/large.xlsx", sheet_name='음식료품')
fou = pd.read_excel("drive/MyDrive/large.xlsx", sheet_name='의류잡화')
fiv = pd.read_excel("drive/MyDrive/large.xlsx", sheet_name='스포츠문화레저')
six = pd.read_excel("drive/MyDrive/large.xlsx", sheet_name='미용')
sev = pd.read_excel("drive/MyDrive/large.xlsx", sheet_name='자동차')

In [ ]:
# 데이터프레임 합치기

all=pd.concat([one,two,thr,fou,fiv,six,sev])
#print(all)
# 데이터 중 대분류만 뽑기

data = all
del data['중분류']
del data['소분류']
print(data)
#len(data)

       대분류        키워드
0    요식/유흥         갈비
1    요식/유흥       곱창구이
2    요식/유흥       꽁치구이
3    요식/유흥       꼬치구이
4    요식/유흥          김
..     ...        ...
495    자동차    아이로드gps
496    자동차  유라이브알바트로스
497    자동차       더보임x
498    자동차  하이패스단말기무선
499    자동차   차량용위치추적기

[24417 rows x 2 columns]


In [ ]:
#konlpy 전처리
data['키워드'].nunique()
data.drop_duplicates(subset=['키워드'], inplace=True)
#print('총 샘플의 수 :',len(data))
data['키워드'] = data['키워드'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
data['키워드'] = data['키워드'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
data['키워드'].replace('', np.nan, inplace=True)
print(data.isnull().sum())
data = data.dropna(how = 'any')
print(len(data))

대분류      0
키워드    143
dtype: int64
23389


In [ ]:
#키워드 토큰화
data_token = [word_tokenize(sent) for sent in data['키워드']]
print('총 샘플의 개수 : {}'.format(len(data_token)))
# 토큰화된 데이터가 카테고라이징이 안된 듯..?

총 샘플의 개수 : 23389


In [ ]:
# train test 분리 (7:3)
from sklearn.model_selection import train_test_split

#x가 키워드
#y는 카테고리
#x=data['키워드']
#y=data['대분류']
#x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=0.3, 
                                                    shuffle=True, 
                                                    random_state=1000)
#print(x_train)
#print(y_train)

In [ ]:
# train test 분리 2

from sklearn.utils import shuffle
data_random = shuffle(data)
#data_random.head(10)

train=data_random[:20000]
#print(len(train))
#print(train)
test=data_random[20000:]
#print(len(test))

20000
             대분류         키워드
3433          미용  나이팅게일클렌징패드
4782          미용     데이비캄유브이
2397       요식/유흥    칼라마르 타바스
1237       음/식료품        염지순살
1575   스포츠/문화/레저     스포츠아일랜드
...          ...         ...
11416         미용    수채화타투스티커
163    스포츠/문화/레저      남성여름조끼
8550          미용    록시땅체리블라썸
3286       음/식료품     달콘초당옥수수
637           미용       퓨어링크림

[20000 rows x 2 columns]
3389


In [ ]:
!pip install fasttext

In [ ]:
df2 = pd.DataFrame(columns=['대분류','키워드'])
df2['대분류']='__label__'+train['대분류']
df2['키워드']=train['키워드']
df2.to_csv('labelingtrain.txt', sep = '\t', index = False)
labeling = pd.read_csv("/content/labelingtrain.txt", sep = '\t')
print(labeling)

                      대분류         키워드
0             __label__미용  나이팅게일클렌징패드
1             __label__미용     데이비캄유브이
2          __label__요식/유흥    칼라마르 타바스
3          __label__음/식료품        염지순살
4      __label__스포츠/문화/레저     스포츠아일랜드
...                   ...         ...
19995         __label__미용    수채화타투스티커
19996  __label__스포츠/문화/레저      남성여름조끼
19997         __label__미용    록시땅체리블라썸
19998      __label__음/식료품     달콘초당옥수수
19999         __label__미용       퓨어링크림

[20000 rows x 2 columns]


In [ ]:
import fasttext

model = fasttext.train_supervised('/content/labelingtrain.txt')
model.save_model("model_1.bin")

In [ ]:
# test dataset의 예측 카테고리
predictions=[]
for line in test['키워드']:
  pred_label=model.predict(line, threshold=0.5)[0]
  predictions.append(pred_label)

print(predictions)

[]


In [ ]:
# 예측 값 예시
model.predict("쭈꾸미삼겹살")

(('__label__음/식료품',), array([0.45012254]))

In [ ]:
user = input("키워드를 입력해주세요 : ")

def model_apply(user):
  #output = model.predict(user)
  output = model.predict(user)[0]
  output2 = str(output)
  #print(output2.replace('_','').replace('label',''))
  output3=output2.replace('_','').replace('label','')
  return output3

result=model_apply(user)
result=result.replace("(","").replace(")","").replace(",","").replace("'","")
print("'{user}'의 카테고리는 '{result}'입니다.".format(user=user,result=result))

키워드를 입력해주세요 : 수분크림
'수분크림'의 카테고리는 '미용'입니다.
